In [23]:
import pandas as pd
import os

import tkinter as tk
from tkinter import ttk
from tkinter import filedialog

from tkinter import messagebox

import re
import time

In [24]:
print('正在打开……')
df_mr=pd.DataFrame()
root = tk.Tk()
root.withdraw()
# path = filedialog.askopenfilename()
# path_out, temp = os.path.split(path)
# file_name, extension = os.path.splitext(temp)
# print('path:', path_out)
# print('file_name:', file_name)
# print('extension:', extension)

datapath1 = filedialog.askopenfilename()

正在打开……


### 读取源数据

In [25]:
df1=pd.read_excel(datapath1,sheet_name='终端购机',dtype='str')
df2=pd.read_excel(datapath1,sheet_name='无宽客户',dtype='str')
df3=pd.read_excel(datapath1,sheet_name='全家享2.0',dtype='str')

### 备份

In [93]:
df1_bck=df1.copy()
df2_bck=df2.copy()
df3_bck=df3.copy()

### 还原

In [94]:
df1=df1_bck
df2=df2_bck
df3=df3_bck

### 读取派单规则

In [26]:
print('正在打开……')
df_mr=pd.DataFrame()
root = tk.Tk()
root.withdraw()
# path = filedialog.askopenfilename()
# path_out, temp = os.path.split(path)
# file_name, extension = os.path.splitext(temp)
# print('path:', path_out)
# print('file_name:', file_name)
# print('extension:', extension)

datapath_r = filedialog.askopenfilename()

正在打开……


In [27]:
df_r1=pd.read_excel(datapath_r,sheet_name='渠道')
df_r2=pd.read_excel(datapath_r,sheet_name='直销')
df_r3=pd.read_excel(datapath_r,sheet_name='三大家')

### 数据清理

将不是数字等转化为数字。

In [95]:
# df1[['服务中心ID','网格ID']] = df1[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# df2[['服务中心ID','网格ID']] = df2[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# df3[['服务中心ID','网格ID']] = df3[['服务中心ID','网格ID']].apply(pd.to_numeric, errors='coerce').fillna(0.0)
# 'raise'参数：无效的解析将引发异常
# 'corece'参数：将无效解析设置为NaN
# 'ignore'参数：无效的解析将返回输入,也就是遇到无效数据不解析，也不会填充NaN或者0
# 用ignore保留中文
df1 = df1.apply(pd.to_numeric, errors='ignore').fillna(0.0)
df2 = df2.apply(pd.to_numeric, errors='ignore').fillna(0.0)
df3 = df3.apply(pd.to_numeric, errors='ignore').fillna(0.0)

### 递归，根据各种不同条件，取满足条件的号码，并依次遍历

实际上就是将get和set改变为装饰器，也就是点访问

df__t_tmp 满足条件的号码池
pdl 派单量
row 打标，分配编码
stack 堆栈，存储满足各种条件的号码池
df_res_total 存放本次循环的结果
df_res 取满足派单量的数据
df_orgin 原始df

In [96]:
#取出各条件号码，依次遍历

def handle(df_t_tmp, pdl, row, queue, df_res_total, df_res,df_orgin):
    # print(len(df_t_tmp))
    # 结束条件
    if (len(df_t_tmp) == 10000):
        return df_res_total

    if ((len(df_t_tmp) >= pdl)):
        df_res = df_t_tmp.head(pdl).copy()
        df_orgin=df_orgin[~df_orgin.号码.isin(df_res.号码)]
        df_res['打标'] = row
        df_res_total = pd.concat([df_res_total, df_res])
        # print(df_res_total)
        return df_res_total
    else:
        last_pdl = pdl - len(df_t_tmp)
        df_res = df_t_tmp.head(len(df_t_tmp)).copy()
        df_orgin=df_orgin[~df_orgin.号码.isin(df_res.号码)]
        df_res['打标'] = row
        df_res_total = pd.concat([df_res_total, df_res])
        # print(df_res_total)
        df_t_tmp = queue.pop(0)  # 队列出队
        return handle(df_t_tmp, last_pdl, row, queue, df_res_total, df_res,df_orgin)

### 渠道

### 无宽客户

条件对应二进制码

In [97]:
term=[15,14,13,11,12,10,9,8,7,6,5,3,4,2,1,0]

条件结构体

In [98]:
class wkkh_term(object):

    def __init__(self, a, b,c,d):

        self.a = int(a)

        self.b = int(b)

        self.c = int(c)
        
        self.d = int(d)

    def df_wk(self,df_tmp):
        df_tmp_res=df_tmp[(df_tmp['80以上资费高价值客户']==self.a)&(df_tmp['我号异宽']==self.b)&(df_tmp['终端换机']==self.c)&(df_tmp['4G资费']==self.d)]
        return df_tmp_res

### 条件进队

In [99]:
def get_wkkh(n,df_tmp):
    df_stack = []
    for i in term:
        res='{:04b}'.format(i)
        a=res[0]
        b=res[1]
        c=res[2]
        d=res[3]
        ob=wkkh_term(a,b,c,d)
        df_res=wkkh_term.df_wk(ob,df_tmp)
        df_stack.append(df_res)
    #结束条件
    df_end = pd.Series(range(0, 10000))
    df_stack.append(df_end)
    return df_stack

无宽客户

In [101]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r1.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()
    df_tmp=df2[df2['网格ID']==row['微网格ID']]
  
    df_tmp_res=pd.DataFrame()   

    df_orgin=df2#修改条件

    pdl1 = row['无宽客户']#修改条件
    df_stack = []
    df_stack = get_wkkh(16,df_tmp)
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    pdl1 = int(pdl1)
    row = row['派单编号']
    #压出队头，利用队列思想
    df_t_tmp=df_stack.pop(0)
    # 递归
    df_res = handle(df_t_tmp, pdl1, row, df_stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
                     
df_res_total_l.to_excel('渠道-无宽客户-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]

全家享2.0

In [38]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r1.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()

    df_tmp=df3[df3['网格ID']==row['微网格ID']]
    df_t_tmp1=df_tmp[(df_tmp['是否裸奔高价值']==1)&(df_tmp['是否质差客户']==1)&(df_tmp['有免费成员空间']==1)]
    df_t_tmp2=df_tmp[(df_tmp['是否裸奔高价值']==1)&(df_tmp['是否质差客户']==1)&(df_tmp['有免费成员空间']==0)]
    df_t_tmp3=df_tmp[(df_tmp['是否裸奔高价值']==1)&(df_tmp['是否质差客户']==0)&(df_tmp['有免费成员空间']==0)]
    df_t_tmp3=df_tmp[(df_tmp['是否裸奔高价值']==0)&(df_tmp['是否质差客户']==0)&(df_tmp['有免费成员空间']==0)]


    df_orgin=df3#修改条件

    pdl1 = row['全家享2.0']#修改条件
    stack = []
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    stack.append(df_end)
    stack.append(df_t_tmp3)
    stack.append(df_t_tmp2)

    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
df_res_total_l.to_excel('渠道-全家享2.0-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]


### 直销

无宽客户

In [41]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r2.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()
    df_tmp=df2[df2['服务中心ID']==row['服务中心ID']]
    df_tmp_res=pd.DataFrame()



    df_orgin=df2#修改条件

    pdl1 = row['无宽客户']#修改条件
    df_stack = []
    df_stack = get_wkkh(16,df_tmp)
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件

    #队列
    df_t_tmp=df_stack.pop(0)

    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp, pdl1, row, df_stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
                     
df_res_total_l.to_excel('直销-无宽客户-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]

全家享

In [43]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r2.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()


    df_tmp=df3[df3['服务中心ID']==row['服务中心ID']]
    df_t_tmp1=df_tmp[(df_tmp['是否裸奔高价值']==1)&(df_tmp['是否质差客户']==1)&(df_tmp['有免费成员空间']==1)]
    df_t_tmp2=df_tmp[(df_tmp['是否裸奔高价值']==1)&(df_tmp['是否质差客户']==1)&(df_tmp['有免费成员空间']==0)]
    df_t_tmp3=df_tmp[(df_tmp['是否裸奔高价值']==1)&(df_tmp['是否质差客户']==0)&(df_tmp['有免费成员空间']==0)]
    df_t_tmp4=df_tmp[(df_tmp['是否裸奔高价值']==0)&(df_tmp['是否质差客户']==0)&(df_tmp['有免费成员空间']==0)]


    df_orgin=df3#修改条件

    pdl1 = row['全家享2.0']#修改条件
    stack = []
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    stack.append(df_end)
    stack.append(df_t_tmp4)
    stack.append(df_t_tmp3)
    stack.append(df_t_tmp2)


    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
df_res_total_l.to_excel('直销-全家享2.0-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]

三大家


终端购机

In [54]:
df_r3

,区县,服务中心,服务中心ID,渠道名称,战略,派单编号,终端购机,全家享2.0
0,长宁,长宁城区服务中心,1277398,宜宾长宁分公司竹都大道营业厅燕君通讯（引商入柜）,燕君,1064424,100,100
1,筠连,筠连城区服务中心,1277422,宜宾筠连分公司城区分局燕君加盟连锁店,燕君,1063880,100,100
2,兴文,兴文城区服务中心,1277414,宜宾兴文分公司城区分局燕君加盟连锁店,燕君,1044729,100,100
3,兴文,兴文城区服务中心,1277414,宜宾兴文分公司城区分局燕君翡翠店,燕君,1436061,100,100
4,高县,文江乡镇服务中心,1670358,宜宾高县分公司城区分局燕君文江加盟连锁店,燕君,1095313,100,100
...,...,...,...,...,...,...,...,...
76,翠屏,翠屏城区服务中心,1277400,宜宾翠屏分公司城区南街营业厅佳海通讯（引商入柜）,佳海,1398744,100,100
77,翠屏,翠屏城区服务中心,1277400,宜宾翠屏分公司城区西郊营业厅佳海通讯（自建他营）,佳海,16462,100,100
78,叙州,柏溪城区服务中心,1277415,宜宾叙州区分公司振兴营业厅佳海通讯（引商入柜）,佳海,69053,100,100
79,叙州,柏溪城北服务中心,1460861,宜宾叙州区分公司城北服务中心凯悦金沙佳海加盟连锁店,佳海,1580971,100,100


In [64]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r3.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()

    df_tmp=df1[df1['服务中心ID']==row['服务中心ID']]
    df_t_tmp1=df_tmp[(df_tmp['是否为宽带客户']=='无宽客户')]
    df_t_tmp2=df_tmp[(df_tmp['是否为宽带客户']=='有宽客户')]


    df_orgin=df1#修改条件

    pdl1 = row['终端购机']#修改条件
    stack = []
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    stack.append(df_end)
    stack.append(df_t_tmp2)

    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
df_res_total_l.to_excel('三大家-终端购机-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]

全家享

In [34]:
df_res_total=pd.DataFrame()
df_res_total_l=pd.DataFrame()
for index, row in df_r3.iterrows():
    #取出各条件号码，依次遍历
    df_res=pd.DataFrame()

    df_tmp=df3[df3['服务中心ID']==row['服务中心ID']]
    df_t_tmp1=df_tmp[(df_tmp['是否裸奔高价值']==1)&(df_tmp['是否质差客户']==1)&(df_tmp['有免费成员空间']==1)]
    df_t_tmp2=df_tmp[(df_tmp['是否裸奔高价值']==1)&(df_tmp['是否质差客户']==1)&(df_tmp['有免费成员空间']==0)]
    df_t_tmp3=df_tmp[(df_tmp['是否裸奔高价值']==1)&(df_tmp['是否质差客户']==0)&(df_tmp['有免费成员空间']==0)]
    df_t_tmp4=df_tmp[(df_tmp['是否裸奔高价值']==0)&(df_tmp['是否质差客户']==0)&(df_tmp['有免费成员空间']==0)]

    df_orgin=df3#修改条件

    pdl1 = row['全家享2.0']#修改条件
    stack = []
    df_end = pd.Series(range(0, 10000))
    # print(len(df_end))
    #从2开始，有n个条件，就压栈n-1个条件和df_end条件
    stack.append(df_end)
    stack.append(df_t_tmp4)    
    stack.append(df_t_tmp3)
    stack.append(df_t_tmp2)

    pdl1 = int(pdl1)
    row = row['派单编号']
    # 递归
    df_res = handle(df_t_tmp1, pdl1, row, stack, df_res_total, df_res,df_orgin)
    df_res_total_l = pd.concat([df_res_total_l, df_res])
df_res_total_l.to_excel('三大家-全家享-结果.xlsx')
df3=df3[~df3.号码.isin(df_res_total_l.号码)]
df2=df2[~df2.号码.isin(df_res_total_l.号码)]
df1=df1[~df1.号码.isin(df_res_total_l.号码)]